In [33]:
import pandas as pd
import seaborn as sns
import traintools
from trainconstants import *
from collections import defaultdict
import networkx as nx

sns.set()
%matplotlib inline

df = pd.read_csv("datasets/nsdata1.txt",  delimiter=r"\s+")
df['departure_time'] = df['departure_time'].map(traintools.to_minutes_past_midnight)
df['arrival_time'] = df['arrival_time'].map(traintools.to_minutes_past_midnight)
df.head()

,train_number,start,end,departure_time,arrival_time,first_class,second_class
0,2123,2,3,420,460,4,58
1,2123,3,4,463,518,14,328
2,2127,1,2,408,475,47,340
3,2127,2,3,481,521,35,272
4,2127,3,4,523,578,19,181


In [34]:
schedule_network= df.loc[:,['first_class', 'second_class']]
schedule_network['start'] = list(zip(df.start, df.departure_time))
schedule_network['end'] = list(zip(df.end, df.arrival_time))

G = nx.from_pandas_edgelist(df=schedule_network, source='start', target='end', edge_attr=['first_class', 'second_class'])